# Create groups of 65 slices

In [39]:
in_path = "D:/SELF-STUDY/freeCodeCamp.org/Kaggle Data/dicom_file/images"
out_path = "D:/SELF-STUDY/freeCodeCamp.org/Kaggle Data/dicom_groups/images"

In [8]:
!pip install glob2

  Created wheel for glob2: filename=glob2-0.7-py2.py3-none-any.whl size=9320 sha256=e847af30ecf4c34b6103365cd5b38a25b287d1206c61e250f12eabf411e90a9e
  Stored in directory: c:\users\tamzi\appdata\local\pip\cache\wheels\95\59\ed\52469ecfae47a78c25f11bcb49db42ff84629df01861876883
Successfully built glob2


In [9]:
!pip install pytest-shutil

  Using cached termcolor-1.1.0-py3-none-any.whl


In [3]:
!pip install nibabel

In [7]:
from glob import glob
import shutil
import os
import dicom2nifti
import nibabel as nib
import numpy as np

In [32]:
patient_list = glob(in_path + '/*')
print(patient_list[0])

D:/SELF-STUDY/freeCodeCamp.org/Kaggle Data/dicom_file/labels\liver_0


In [40]:
# glob returns a list of directories for each file inside mother folder 

for patient in glob(in_path + '/*'):                           # everything inside the folder
    patient_name = os.path.basename(os.path.normpath(patient)) # normalizes path # Basename returns the last item of the path
    number_folders = int(len(glob(patient + '/*'))/64)   #total slices for the patient / lowest # slices a patient has
    
    # groups
    for i in range(number_folders):                  #parses through all groups
        output_path_name = os.path.join(out_path, patient_name + '_' + str(i))    #joins like path1/path2
        os.mkdir(output_path_name)               #create folder
        #print(output_path_name)
        
        # slices
        for j, file in enumerate(glob(patient + '/*')):           # parses through all slices # file has path of each dicom slice
            if j > 64:                                            # min slices
                break
            shutil.move(file, output_path_name)                   # moves slices

# Convert Dicom files into Nifties

In [42]:
!pip install dicom2nifti

In [50]:
in_path_images = 'D:/SELF-STUDY/freeCodeCamp.org/Kaggle Data/dicom_groups/images/*'
in_path_labels = 'D:/SELF-STUDY/freeCodeCamp.org/Kaggle Data/dicom_groups/labels/*'
out_path_images = 'D:/SELF-STUDY/freeCodeCamp.org/Kaggle Data/nifti_files/images'
out_path_labels = 'D:/SELF-STUDY/freeCodeCamp.org/Kaggle Data/nifti_files/labels'

list_images = glob(in_path_images)
list_labels = glob(in_path_labels)

In [47]:
list_images

['D:/SELF-STUDY/freeCodeCamp.org/Kaggle Data/dicom_groups/images\\liver_0_0',
 'D:/SELF-STUDY/freeCodeCamp.org/Kaggle Data/dicom_groups/images\\liver_1_0']

In [49]:
list_labels

['D:/SELF-STUDY/freeCodeCamp.org/Kaggle Data/dicom_groups/labels\\liver_0_0',
 'D:/SELF-STUDY/freeCodeCamp.org/Kaggle Data/dicom_groups/labels\\liver_1_0']

In [52]:
for patient in list_images: 
    patient_name = os.path.basename(os.path.normpath(patient))    # patient name
    dicom2nifti.dicom_series_to_nifti(patient, os.path.join(out_path_images, patient_name + '.nii.gz'))       # runs for each path

In [53]:
for patient in list_labels: 
    patient_name = os.path.basename(os.path.normpath(patient))    # patient name
    dicom2nifti.dicom_series_to_nifti(patient, os.path.join(out_path_labels, patient_name + '.nii.gz'))       # runs for each path

# Find empty slices

In [11]:
# if pixel values are more then 1, then means contains label
# if pixel valyus are zero, means only contains background

input_nifti_file_path = 'D:/SELF-STUDY/freeCodeCamp.org/Kaggle Data/nifti_files/labels/*'
list_labels = glob(input_nifti_file_path)     # list of addresses of all patients

for patient in list_labels:   
    nifti_file = nib.load(patient)            # label of each patient   
    fdata = nifti_file.get_fdata()            # gets frame data in an array    
    np_unique = np.unique(fdata)                
    if len(np_unique) == 1:                   # all zero in array means slice picture contains only background
        print(patient)                        # prints that path
        #shutil.rmtree(patient)               # remove that slice